In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# orig_url = "https://drive.google.com/file/d/12fFZ9k8wsmWBVUhcsVxmKsqHxaVzAzqt/view?usp=sharing"
# file_id = orig_url.split('/')[-2]
# data_path='https://drive.google.com/uc?export=download&id=' + file_id

data_path='https://drive.google.com/uc?export=download&id=1A3MUldrs0z09DlYR6Y1utfySwKNO9Qsz'
df = pd.read_csv(data_path)
df.head(10)

,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,4528,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No
2,6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.5,No
4,432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.5,No
5,2215,4312-GVYNH,Female,0,Yes,No,70,No,No phone service,DSL,...,Yes,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),49.85,3370.2,No
6,5260,2495-KZNFB,Female,0,No,No,33,Yes,Yes,Fiber optic,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,90.65,2989.6,No
7,6001,4367-NHWMM,Female,0,No,No,1,No,No phone service,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,24.90,24.9,No
8,1480,8898-KASCD,Male,0,No,No,39,No,No phone service,DSL,...,Yes,Yes,No,No,One year,No,Mailed check,35.55,1309.15,No
9,5137,8016-NCFVO,Male,1,No,No,55,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,116.50,6382.55,No


In [ ]:
df["SeniorCitizen"] = df["SeniorCitizen"].astype("object")
df["TotalCharges"]= df["TotalCharges"].apply(lambda x: "0" if x == " " else x )
df["TotalCharges"] = df["TotalCharges"].astype("float64")
df.dtypes

Unnamed: 0            int64
customerID           object
gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [ ]:
df["num_services"] = (
    df["PhoneService"].apply(lambda x: 1 if x == "Yes" else 0 )+
    df["MultipleLines"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["InternetService"].apply(lambda x: 0 if x == "No" else 1 ) +
    df["OnlineSecurity"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["OnlineBackup"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["DeviceProtection"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["TechSupport"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["StreamingTV"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["StreamingMovies"].apply(lambda x: 1 if x == "Yes" else 0 )
)
df["num_services"]

0       2
1       5
2       3
3       3
4       4
       ..
5981    5
5982    9
5983    1
5984    6
5985    1
Name: num_services, Length: 5986, dtype: int64

In [ ]:
df["auto_payment"] = df["PaymentMethod"].apply(lambda x: 1 if "automatic" in x  else 0 )
df["auto_payment"] = df["auto_payment"].astype("object")

In [ ]:
X = df.drop(columns = ['Unnamed: 0','Churn', "customerID", 'auto_payment', 'num_services'])
y = df['Churn']

In [ ]:
X.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [ ]:
#Added number of services and removed individual services
X_num_services = df.drop(columns=['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'auto_payment', 'TotalCharges', 'Unnamed: 0','Churn', "customerID"])
X_num_services.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5986 entries, 0 to 5985
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5986 non-null   object 
 1   SeniorCitizen     5986 non-null   object 
 2   Partner           5986 non-null   object 
 3   Dependents        5986 non-null   object 
 4   tenure            5986 non-null   int64  
 5   Contract          5986 non-null   object 
 6   PaperlessBilling  5986 non-null   object 
 7   PaymentMethod     5986 non-null   object 
 8   MonthlyCharges    5986 non-null   float64
 9   num_services      5986 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 467.8+ KB


In [ ]:
# Removed Payment method
X_auto = df.drop(columns=['PaymentMethod', 'num_services', 'TotalCharges','Churn','customerID','Unnamed: 0'])
X_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5986 entries, 0 to 5985
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5986 non-null   object 
 1   SeniorCitizen     5986 non-null   object 
 2   Partner           5986 non-null   object 
 3   Dependents        5986 non-null   object 
 4   tenure            5986 non-null   int64  
 5   PhoneService      5986 non-null   object 
 6   MultipleLines     5986 non-null   object 
 7   InternetService   5986 non-null   object 
 8   OnlineSecurity    5986 non-null   object 
 9   OnlineBackup      5986 non-null   object 
 10  DeviceProtection  5986 non-null   object 
 11  TechSupport       5986 non-null   object 
 12  StreamingTV       5986 non-null   object 
 13  StreamingMovies   5986 non-null   object 
 14  Contract          5986 non-null   object 
 15  PaperlessBilling  5986 non-null   object 
 16  MonthlyCharges    5986 non-null   float64


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, r2_score, mean_squared_error, f1_score


from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTENC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from imblearn.pipeline import Pipeline as imbpipeline


In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
categorical_features, numerical_features

# added smote


# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='constant', fill_value=0), numerical_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])


# Use SMOTENC for handling imbalanced target
categorical_indices = [X.columns.get_loc(col) for col in categorical_features]

pipeline = imbpipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTENC(categorical_features=categorical_indices, random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print('all features\n')
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')



all features

Accuracy: 0.7867483296213809
Confusion Matrix:
[[1147  173]
 [ 210  266]]
Classification Report:
              precision    recall  f1-score   support

          No       0.85      0.87      0.86      1320
         Yes       0.61      0.56      0.58       476

    accuracy                           0.79      1796
   macro avg       0.73      0.71      0.72      1796
weighted avg       0.78      0.79      0.78      1796



### hyperparameter tuning for the RandomForestClassifier using GridSearchCV:

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parameters for GridSearchCV
param_grid = {
    'classifier__n_estimators': [50],
    'classifier__max_depth': [None, 10],
    'classifier__min_samples_split': [2],
    'classifier__min_samples_leaf': [4]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best model from grid search
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict using the best model
y_pred_best = best_model.predict(X_test)

# Evaluate the best model's performance
best_accuracy = accuracy_score(y_test, y_pred_best)
best_conf_matrix = confusion_matrix(y_test, y_pred_best)
best_class_report = classification_report(y_test, y_pred_best)

print(f'Best Model Accuracy: {best_accuracy}')
print(f'Best Model Confusion Matrix:\n{best_conf_matrix}')
print(f'Best Model Classification Report:\n{best_class_report}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan]
  warnings.warn(


Best Model Accuracy: 0.7761692650334076
Best Model Confusion Matrix:
[[1089  231]
 [ 171  305]]
Best Model Classification Report:
              precision    recall  f1-score   support

          No       0.86      0.82      0.84      1320
         Yes       0.57      0.64      0.60       476

    accuracy                           0.78      1796
   macro avg       0.72      0.73      0.72      1796
weighted avg       0.79      0.78      0.78      1796



In [ ]:
# all features to be tried
X_list=[X,X_num_services,X_auto]

In [ ]:
# Define your preprocessor and pipeline (excluding the train-test split and fit)
def create_pipeline(categorical_features, numerical_features):
    # Create the column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', SimpleImputer(strategy='constant', fill_value=0), numerical_features),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))
            ]), categorical_features)
        ])

    # Identify the categorical indices for SMOTENC
    categorical_indices = [list(X_list[0].columns).index(col) for col in categorical_features]

    # Create the pipeline
    pipeline = imbpipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTENC(categorical_features=categorical_indices, random_state=42)),
        ('classifier', RandomForestClassifier(random_state=42,class_weight='balanced'))
    ])

    return pipeline

In [ ]:
# Define the hyperparameter grid (if you plan to perform GridSearchCV for each feature set)
param_grid = {
    'classifier__n_estimators': [50],
    'classifier__max_depth': [None],                    #, 10, 20, 30
    'classifier__min_samples_split': [2],
    'classifier__min_samples_leaf': [2]
}

In [ ]:
# Iterate over each feature set
for i, X_features in enumerate(X_list):
    print(f"Training on feature set {i+1}")

    # Update categorical and numerical features based on current feature set
    categorical_features = X_features.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_features = X_features.select_dtypes(include=['int', 'float']).columns.tolist()

    # Split your data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.3, random_state=42, stratify=y)

    # Create the pipeline with the current feature set
    pipeline = create_pipeline(categorical_features, numerical_features)

    # Optional: Perform grid search for hyperparameter tuning
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Get the best model from grid search
    best_model = grid_search.best_estimator_

    # Predict using the best model
    y_pred_best = best_model.predict(X_test)

    # Evaluate the best model's performance
    best_accuracy = accuracy_score(y_test, y_pred_best)
    best_conf_matrix = confusion_matrix(y_test, y_pred_best)
    best_class_report = classification_report(y_test, y_pred_best)

    # Print the results for the current feature set
    print(f'Best Model Accuracy for feature set {i+1}: {best_accuracy}')
    print(f'Best Model Confusion Matrix for feature set {i+1}:\n{best_conf_matrix}')
    print(f'Best Model Classification Report for feature set {i+1}:\n{best_class_report}')
    print('\n' + '-'*80 + '\n')

Training on feature set 1


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan]
  warnings.warn(


Best Model Accuracy for feature set 1: 0.7761692650334076
Best Model Confusion Matrix for feature set 1:
[[1089  231]
 [ 171  305]]
Best Model Classification Report for feature set 1:
              precision    recall  f1-score   support

          No       0.86      0.82      0.84      1320
         Yes       0.57      0.64      0.60       476

    accuracy                           0.78      1796
   macro avg       0.72      0.73      0.72      1796
weighted avg       0.79      0.78      0.78      1796


--------------------------------------------------------------------------------

Training on feature set 2


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan]
  warnings.warn(


Best Model Accuracy for feature set 2: 0.7544543429844098
Best Model Confusion Matrix for feature set 2:
[[1047  273]
 [ 168  308]]
Best Model Classification Report for feature set 2:
              precision    recall  f1-score   support

          No       0.86      0.79      0.83      1320
         Yes       0.53      0.65      0.58       476

    accuracy                           0.75      1796
   macro avg       0.70      0.72      0.70      1796
weighted avg       0.77      0.75      0.76      1796


--------------------------------------------------------------------------------

Training on feature set 3


ValueError: ignored